In [1]:
# https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/
# https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/

# README https://www.ncbi.nlm.nih.gov/variation/docs/human_variation_vcf/

### Set up parameters

In [2]:
# !gzip -dk "../data/clinvar.vcf.gz"
# !gzip -dk "../data/00-All.vcf.gz"

In [21]:
# vcf_path = "../data/00-common_all.vcf.gz"
# vcf_path = "../data/00-common_all.vcf"
vcf_path = "../data/clinvar.vcf"
# vcf_path = "../data/00-All.vcf"

# output_name = "output_snps" 
output_name = 'output_clinvar_snps'
# output_name = 'output_all_snps'

vcf_dataframe_path = "../data/clinvar.pkl"

snps_subset = None
# snps_subset = 100

if snps_subset != None:
    output_bed_path = "../per_RBP_BED/" + output_name + "SMALL" + ".bed"  # Path to the output BED file
else:
    output_bed_path = "../per_RBP_BED/" + output_name + ".bed"  # Path to the output BED file


### Definiton of methods

In [4]:

# Load the dataset
# df = pd.read_csv('../data/gwas-catalog-v1.0.3-studies-r2023-08-26.tsv', sep='\t', nrows=10)  # Adjust the separator if needed
# df.columns

# print(df['CHR_ID'])
# print(df['CHR_POS'])
# print(df['SNPS'])
# print(df['STRONGEST SNP-RISK ALLELE'])
# print(df['CONTEXT'])

In [5]:
import pysam

# Load the VCF file
vcf_file = pysam.VariantFile(vcf_path)

# Get the chromosome names
# vcf_chromosomes = list(vcf_file.header.contigs.keys())

In [9]:
import pandas as pd

def vcf_to_pandas(file_path, num_data_lines=None):
    header=None
    lines = []
    with open(file_path, 'r') as vcf_file:
        for line in vcf_file:
            # If the line is a header line, print it
            if line.startswith("#"):
                line_striped = line.strip('#')
                if not line_striped.startswith('INFO'):
                    if not '=' in line_striped:
                        # print(line_striped)
                        header = line_striped.replace('\n','').split('\t')
            # If the line is a data line, print up to the specified number of data lines
            else:
                if num_data_lines == None:
                    lines.append(line.split('\t'))
                else:
                    if num_data_lines > 0:
                        # print(line)
                        lines.append(line.split('\t'))
                        num_data_lines -= 1
                    else:
                        break
    print(header)
    # _ = [print(x) for x in lines]
    return pd.DataFrame(lines, columns=header)

cvf_df = vcf_to_pandas(vcf_path)


['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']


In [22]:
cvf_df.to_pickle(vcf_dataframe_path)

In [23]:
def count_records_in_vcf(vcf_path):
    count = 0
    with open(vcf_path, 'r') as vcf_file:
        for line in vcf_file:
            if not line.startswith("#"):
                count += 1
    return count

num_records = count_records_in_vcf(vcf_path)
print(f"Number of records in the VCF file: {num_records}")

Number of records in the VCF file: 2256164


In [24]:
# basic SNP genomic coordinates extraction 

def extract_snps_regions(vcf_path, bed_path, subset=None, flank_length=200):
    i = 0
    with open(vcf_path, 'r') as vcf_file, open(bed_path, 'w') as bed_file:
        for line in vcf_file:
            if not line.startswith("#"):  # Ignore header lines
                columns = line.strip().split('\t')
                chrom = 'chr' + columns[0]
                pos = int(columns[1])
                ref = columns[3]
                alt = columns[4]
                name = columns[2]  # Variant identifier
                score = columns[5] if columns[5] != "." else "0"  # Quality of the variant call
                strand = "+"  # Default strand value
                ref_alt = f"{ref}/{alt}"  # REF/ALT column
                info = columns[7]  # info column
                # GENEINFO

                # Check if the variant is a SNP
                if len(ref) == 1 and len(alt) == 1 and ref != '.' and alt != '.':
                    start = pos - flank_length
                    end = pos + flank_length
                    # bed_file.write(f"{chrom}\t{start}\t{end}\t{name}\t{score}\t{strand}\n")
                    # bed_file.write(f"{chrom}\t{start}\t{end}\t{name}\t{score}\t{strand}\t{ref_alt}\n")
                    bed_file.write(f"{chrom}\t{start}\t{end}\t{name}\t{score}\t{strand}\t{ref_alt}\t{info}\n")
                
                if subset != None:
                    if i > subset:
                        break
                else:
                    if i % 1000000 == 0:
                        print(i)
                i+=1
                    


### Run the extraction

In [25]:
extract_snps_regions(vcf_path, output_bed_path, subset=snps_subset)


0
1000000
2000000


In [26]:
output_bed_path

'../per_RBP_BED/output_clinvar_snps.bed'

In [28]:
# Load the BED file into a pandas DataFrame
df = pd.read_csv(output_bed_path, sep='\t', header=None, names=['Chromosome', 'Start', 'End', 'Name', 'Score', 'Strand', 'ref_alt'])

# Display the first few rows
print(df.head(2000)[df['Chromosome'] == 'chr1'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Load the BED file into a pandas DataFrame
df = pd.read_csv(output_bed_path, sep='\t', header=None, names=['Chromosome', 'Start', 'End', 'Name', 'Score', 'Strand', 'ref_alt'])

In [ ]:
# For vcf_path = "../data/00-common_all.vcf" - Display the boarder of chr1 and others 
print(df[1000000:10000100])